In [6067]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [6068]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6069]:
def prepare_data(file_name='./train.csv'):
    train_df = pd.read_csv(file_name)
    train_df.drop(train_df.columns[0],axis=1,inplace=True)
    train_npa = train_df.to_numpy()
    
    train_data = []
    max_item_id = 0
    max_user_id = train_npa.shape[0]

    for i, docs in enumerate(train_npa):
        tmp_arr = np.array(list(map(int, list(docs)[0].split())))
        tmp_max = np.max(tmp_arr)
        if tmp_max > max_item_id:
            max_item_id = tmp_max
        train_data.append(tmp_arr)

    train_data = np.array(train_data, dtype=object)

    return train_data, max_item_id, max_user_id

In [6070]:
def create_rating_matrix(train_data, max_item_id, max_user_id):
    R = np.zeros((max_item_id, max_user_id))
    
    for i in range(max_item_id):
        for j in range(max_user_id):
            if i in train_data[j]:
                # user j interacted with item i
                R[i][j] = 1
    
    return R

In [6071]:
def bce_loss(data, i, j):
    hor_sum = np.sum(data, axis=1)
    vert_sum = np.sum(data, axis=0)
    
    return -(data[i, j]*np.ln(hor_sum[i]) + (1-data[i, j])*np.ln(1-vert_sum[j]))

In [6072]:
# def gradient_descent(start, learn_rate=0.01, n_iter=50, tolerance=1e-06):
#     vector = start
#     for _ in range(n_iter):
# #         diff = -learn_rate * np.gradient(vector)
#         if np.all(np.abs(diff) <= tolerance):
#             break
#         vector += diff
#     return vector

In [6073]:
def count_occurrences(row):
    return len([x for x in row if x == 1])

In [6074]:
# def gradient_descent(R, P, Q, max_iter=1, learn_rate=0.01, n_iter=50, tolerance=1e-06, lamb1=0.001, lamb2=0.001):
#     for iters in range(max_iter):
#         print("iteration: {}".format(iters))
#         for x in range(R.shape[1]):
#             for i in range(R.shape[0]):
# #                 o = count_occurrences()
# #                 eps = R[i, x] - (np.dot(Q[i], P[x]))
# #                 eps = R[i, x] - (np.log(sigmoid(Q[i])) + np.log(sigmoid(P[x])))
#                 p = sigmoid(np.dot(Q[i], P[x]))
# #                 print(p)
#                 if math.isnan(p):
#                     print("({}, {}) --- Q[i]: {}, P[x]: {}".format(x, i, Q[i], P[x]))
#                 if p == 0:
#                     p = 0.01
#                 if (1-p) <= 0:
#                     p = 0.99
#                 y = R[i, x]
# #                 print(p)
                
#                 try:
#                     eps = -(np.sum(y*np.log(p) + (1-y)*(np.log(1-p))))
# #                     print(eps)
#                     if math.isinf(eps) or math.isnan(eps):
#                         print("eps is wrong --- y: {}, p: {}".format(y, p))
#                 except:
#                     print("p: {}".format(p))
#                     print("y: {}".format(y))
# #                 eps = -(R[i, x]*np.log(pred) + (1-R[i, x])*np.log((1-pred)))

#                 if (y-p) <= 0:
#                     eps = -eps
#                 Q[i] = Q[i] + learn_rate*(eps*P[x]-lamb2*Q[i])
# #                 print(learn_rate*(eps*Q[i]-lamb1*P[x]))
#                 P[x] = P[x] + learn_rate*(eps*Q[i]-lamb1*P[x])
    
#     return P, Q

In [6075]:
def gradient_descent(R, P, Q, max_iter=15, learn_rate=0.001, lamb1=0.001, lamb2=0.001):
    for iters in range(max_iter):
        print("iteration: {}".format(iters))
        
        r = np.random.randint(0, Q.shape[0])
        for i in range(P.shape[0]):
            y = R[r, i]
            p = np.dot(Q[r], P[i])
            eps = (-y/p)+((1-y)/(1-p))
            eps = np.sign(p)*eps

            Q[r] = Q[r] + learn_rate*(-eps*P[i]-lamb2*Q[r])
            P[i] = P[i] + learn_rate*(-eps*Q[r]-lamb1*P[i])
            
            
            
        r = np.random.randint(0, P.shape[0])
        for i in range(Q.shape[0]):
#             eps = R[i, r] - (np.dot(Q[i], P[r]))
            y = R[i, r]
            p = np.dot(Q[i], P[r])
            eps = (-y/p)+((1-y)/(1-p))
            
            eps = np.sign(p)*eps
    
            Q[i] = Q[i] + learn_rate*(-eps*P[r]-lamb2*Q[i])
            P[r] = P[r] + learn_rate*(-eps*Q[i]-lamb1*P[r])
            
#             print("before: y: {} p: {}, after: {}".format(y, p, np.dot(Q[i], P[r])))
    
    return P, Q

In [6076]:
# def gradient_descent(R, P, Q, max_iter=1, learn_rate=0.1, tolerance=1e-06, lamb1=0.01, lamb2=0.01):
#     for iters in range(max_iter):
#         for x in range(R.shape[1]):
#             for i in range(R.shape[0]):
        
#                 y = R[i, x]
#                 p = np.dot(Q[i], P[x])
# #                 print("y: {}, p: {}".format(y, p))
# #                 if p > 1:
# #                     print(p)
# #                 if p >= 1:
# #                     p = 0.99
# #                 if p < 0:
# #                     p = 0.99

#                 eps = (-y/p)+((1-y)/(1-p))
# #                 eps = np.sign(eps)
# #                 print(eps)
#                 eps = np.sign(p)*eps
#                 Q[i] = Q[i] + learn_rate*(-eps*P[x]-lamb2*Q[i])
#                 P[x] = P[x] + learn_rate*(-eps*Q[i]-lamb1*P[x])

# #                 print("before: y: {} p: {}, after: {}".format(y, p, np.dot(Q[i], P[x])))
                
#     return P, Q

In [6077]:
def get_most_relevant(mat, n=50):
    res = []
    
    for i in range(mat.shape[0]):
        d = dict(enumerate(mat[i]))
        sl = sorted(d.items(), key=lambda item: item[1], reverse=True)
        res.append(sl[:n])

    return res

In [6078]:
def output_result_csv(rel_items):        
#     res = pd.DataFrame.from_dict(rel_items, orient='index', columns=['ItemId'])
    for k, v in rel_items.items():
        rel_items[k] = ' '.join(str(x) for x in v)
        
    res = pd.DataFrame([{'ItemId': v} for v in rel_items.values()], index=rel_items.keys())
    res.index.name = 'UserId'
    
    res.to_csv('./submission.csv')  
        

In [6079]:
def _sample(self, n_users, n_items, indices, indptr):
        """sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_neg_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_users = np.random.choice(
            n_users, size = self.batch_size, replace = False)

        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user]:indptr[user + 1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)

            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items

In [6080]:
test = np.array([1, 0, 0, 1, 0, 1])
itemindex = np.where(test == 1)[0]
itemindex

array([0, 3, 5], dtype=int64)

In [6081]:
def bpr_sgd(R, max_iter=10, learn_rate=0.001, lamb=0.001):
#     Q = np.random.uniform(0, 1, size=(R.shape[1], 16))
#     P = np.random.uniform(0, 1, size=(R.shape[0], 16))

    n_users = R.shape[1]
    n_items = R.shape[0]
    batch_size = 100
    n_factors = 20

    batch_iters = n_users // batch_size

    rstate = np.random.RandomState(1234)
    Q = rstate.normal(size = (R.shape[1], n_factors))
    P = rstate.normal(size = (R.shape[0], n_factors))
    
    for it in range(max_iter):
        for _ in range(batch_iters):
            sampled_pos_items = np.zeros(batch_size, dtype = np.int)
            sampled_neg_items = np.zeros(batch_size, dtype = np.int)
            sampled_users = np.random.choice(n_users, size = batch_size, replace = False)
            
            for idx, user in enumerate(sampled_users):
                pos_items = np.where(np.array(R[:, user]) == 1)[0]
                pos_item = np.random.choice(pos_items)
                
                neg_items = np.where(np.array(R[:, user]) == 0)[0]
                neg_item = np.random.choice(neg_items)
#                 neg_item = np.random.choice(n_items)
#                 while neg_item in pos_items:
#                     neg_item = np.random.choice(n_items)

                sampled_pos_items[idx] = pos_item
                sampled_neg_items[idx] = neg_item
#             print(sampled_pos_items)
#             print(sampled_neg_items)
#             u = np.random.randint(0, R.shape[1])
#             i = np.random.randint(0, R.shape[0])
#             j = np.random.randint(0, R.shape[0])
            u = sampled_users
            i = sampled_pos_items
            j = sampled_neg_items
            
            xuij = np.sum(Q[u] * (P[i] - P[j]), axis = 1)

            sig = np.exp(-xuij) / (1.0 + np.exp(-xuij))
            sig_t = np.tile(sig, (n_factors, 1)).T

            grad_u = sig_t * (P[j] - P[i]) + lamb * Q[u]
            grad_i = sig_t * -Q[u] + lamb * P[i]
            grad_j = sig_t * Q[u] + lamb * P[j]
            Q[u] -= learn_rate * grad_u
            P[i] -= learn_rate * grad_i
            P[j] -= learn_rate * grad_j
    
    return P, Q

In [6082]:
# def partial_BPR(x_uij, partial_x):
#     exp_x = np.exp(-x_uij)
#     return exp_x / (1 + exp_x) * partial_x

In [6083]:
# def bpr_sgd(R, iteration=100, lr=0.01, rr=0.01, k=16):
#     W = np.random.uniform(0, 1, size=(R.shape[1], k))
#     H = np.random.uniform(0, 1, size=(R.shape[0], k))
    
#     for itr in range(iteration):
# #         u, i, j = ds[np.random.randint(len(ds))]
#         u = np.random.randint(0, R.shape[1])
#         i = np.random.randint(0, R.shape[0])
#         j = np.random.randint(0, R.shape[0])
            
# #         x_uij = predict_diff(u, i, j)
#         x_uij = np.sum(W[u] * (H[i]-H[j]))

#         for f in range(k):
#             W[u][f] -= lr * (partial_BPR(x_uij, H[i][f] - H[j][f]) + rr * W[u][f])
#             H[i][f] -= lr * (partial_BPR(x_uij, W[u][f]) + rr * H[i][f])
#             H[j][f] -= lr * (partial_BPR(x_uij, -W[u][f]) + rr * H[f][f])
        
# #         if itr % 10000 == 0:
# #             print(W[18])
# #             print(H[0])
            
#     return W, H

In [6084]:
#         # Q (W)
#         d = (P[i]-P[j])
#         Q[u] = Q[u] + learn_rate*(((np.exp(-xuij))/(1+np.exp(-xuij))) * d + lamb*Q[u])
        
#         # Pi (H)
#         d = -Q[u]
#         P[i] = P[i] + learn_rate*(((np.exp(-xuij))/(1+np.exp(-xuij))) * d + lamb*P[i])
        
#         # Pj (H)
#         d = Q[u]
#         P[j] = P[j] + learn_rate*(((np.exp(-xuij))/(1+np.exp(-xuij))) * d + lamb*P[j])

In [6085]:
train_data, max_item_id, max_user_id = prepare_data()

In [6086]:
# R = create_rating_matrix(train_data, max_item_id, max_user_id)

In [6087]:
M, N = R.shape[0], R.shape[1]

In [6088]:
# u, s, vh = np.linalg.svd(R, )

# # P = u
# # Q = np.matmul(s, vh)

In [6089]:
# P = np.matmul(u, s)
# Q = vh

In [6090]:
# initial_Q = np.random.uniform(0, 1, size=(M, 2))
# initial_P = np.random.uniform(0, 1, size=(N, 2))

In [6091]:
# initial_Q = np.random.uniform(0, 1, size=(M, 2))
# initial_P = np.random.uniform(0, 1, size=(N, 2))
# P, Q = gradient_descent(R, initial_P, initial_Q)

In [6092]:
P, Q = bpr_sgd(R)

In [6093]:
M = np.matmul(Q, np.transpose(P))

In [6094]:
# ids = np.argpartition(M, -50)[-50:]
# best_ids = np.argsort(M[ids])[::-1]
# best = ids[best_ids]

In [6095]:
relevant_items = get_most_relevant(M)
rel_items = [[x[0] for x in l] for l in relevant_items]

In [6096]:
rel_items_dict = dict(enumerate(rel_items))

In [6097]:
output_result_csv(rel_items_dict)

In [6098]:
def calculate_overlap():
    tdf = pd.read_csv('train.csv')
    sdf = pd.read_csv('submission.csv')
    
    return tdf, sdf

In [6099]:
tdf, sdf = calculate_overlap()

In [6100]:
train_npa = tdf.to_numpy()
sub_npa = sdf.to_numpy()

In [6101]:
ta = []
for idx, r in enumerate(train_npa):
    ta.append(np.array(train_npa[idx][1].split(' '), dtype=np.int))
ta = np.array(ta, dtype=object)

sa = []
for idx, r in enumerate(sub_npa):
    sa.append(np.array(sub_npa[idx][1].split(' '), dtype=np.int))
sa = np.array(sa, dtype=object)

In [6102]:
isec = []
for i in range(len(ta)):
    isec.append(np.intersect1d(ta[i], sa[i]).shape[0]/len(ta[i]))

In [6103]:
np.mean(np.array(isec))

0.014776789265974382